In [1]:
import torch
import sklearn
from sklearn.model_selection import train_test_split
import datasets
from transformers import AutoTokenizer, GPT2Tokenizer,  GPT2ForSequenceClassification, Trainer, TrainingArguments
import random
import numpy as np
from datasets import load_dataset


In [2]:
dataset = load_dataset('csv', data_files='..\data\dataset\processed\clean_data_gpt2.csv')

In [3]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['prompt', 'essay', 'label'],
        num_rows: 9766
    })
})


In [4]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

def tokenize_function(examples):
    combined_text = examples["prompt"] + '\n' + examples["essay"]
    return tokenizer(combined_text, padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=False)

In [5]:
encoded_dataset = tokenized_datasets.rename_column("label", "labels")
encoded_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [6]:
small_train_dataset = encoded_dataset["train"].shuffle(seed=42).select(range(5000))
small_eval_dataset = encoded_dataset["train"].shuffle(seed=42).select(range(5000, 7000))
small_test_dataset = encoded_dataset["train"].shuffle(seed=42).select(range(7000, 9766))


In [7]:
print(small_train_dataset)

Dataset({
    features: ['prompt', 'essay', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 5000
})


In [8]:
from transformers import DistilBertForSequenceClassification

# Specify the number of labels in your dataset
num_labels = 12  # Change this based on your dataset
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=num_labels)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=50,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
)

In [14]:
trainer.train()


  0%|          | 0/15650 [00:00<?, ?it/s]

  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 2.7799417972564697, 'eval_runtime': 10.2395, 'eval_samples_per_second': 195.322, 'eval_steps_per_second': 12.208, 'epoch': 1.0}
{'loss': 1.2894, 'grad_norm': 16.68935775756836, 'learning_rate': 1.9361022364217256e-05, 'epoch': 1.6}


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 2.8020873069763184, 'eval_runtime': 10.2872, 'eval_samples_per_second': 194.416, 'eval_steps_per_second': 12.151, 'epoch': 2.0}


...

  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 8.053009986877441, 'eval_runtime': 10.1931, 'eval_samples_per_second': 196.211, 'eval_steps_per_second': 12.263, 'epoch': 49.0}
{'loss': 0.0036, 'grad_norm': 0.04660388082265854, 'learning_rate': 1.9169329073482428e-07, 'epoch': 49.52}


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 8.074740409851074, 'eval_runtime': 10.1442, 'eval_samples_per_second': 197.157, 'eval_steps_per_second': 12.322, 'epoch': 50.0}
{'train_runtime': 4432.7932, 'train_samples_per_second': 56.398, 'train_steps_per_second': 3.531, 'train_loss': 0.2332743097037172, 'epoch': 50.0}


TrainOutput(global_step=15650, training_loss=0.2332743097037172, metrics={'train_runtime': 4432.7932, 'train_samples_per_second': 56.398, 'train_steps_per_second': 3.531, 'total_flos': 3.3122755584e+16, 'train_loss': 0.2332743097037172, 'epoch': 50.0})

In [15]:
from sklearn.metrics import accuracy_score

# Run predictions on the test dataset
predictions = trainer.predict(small_test_dataset)
preds = predictions.predictions.argmax(-1)

# Calculate accuracy
labels = small_test_dataset["labels"]
accuracy = accuracy_score(labels, preds)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

  0%|          | 0/173 [00:00<?, ?it/s]

Test Accuracy: 22.74%
